In [130]:
# encoding:utf-8
import pandas as pd
import lightgbm as lgb
import numpy as np
from sklearn.metrics import log_loss
import sys
import sklearn
from sklearn.model_selection import cross_val_score
from data_processing import further_processing_on_train, further_processing_on_test, basic_processing
from sklearn import preprocessing
import math
import seaborn as sns
import matplotlib.pyplot as plt

In [131]:
lbl = preprocessing.LabelEncoder()

In [132]:
data = pd.concat([pd.read_csv("train.csv", sep=" "), pd.read_csv("test.csv", sep=" ")])
data.drop_duplicates(inplace=True)

In [133]:
train = data[data.is_trade.notnull()]

In [134]:
test = data[data.is_trade.isnull()]
train = basic_processing(train, lbl)
test = basic_processing(test, lbl)

basic processing on time features
basic processing on user features
basic processing on item features
basic processing on shop features
basic processing on context features
basic processing on time features
basic processing on user features
basic processing on item features
basic processing on shop features
basic processing on context features


In [135]:
#sns.pointplot(x='item_collected_level', y='is_trade', data=train)
#plt.ylabel('is_trade', fontsize=12)
#plt.xlabel('shop_score_delivery', fontsize=12)
#plt.xticks(rotation='vertical')

In [140]:
train_off = train.loc[train.day <= 23]
test_off = train.loc[train.day > 23].sample(frac=0.7, random_state=20)

In [141]:
train_off = further_processing_on_train(train_off)
test_off = further_processing_on_test(test_off, train_off)

In [160]:
features = sorted([f for f in list(train_off) if f not in [
    'instance_id', 
    'item_category_list', 
    'item_property_list', 
    'context_id', 
    "context_page_id",
    'context_timestamp', 
    'predict_category_property', 
    'is_trade', 
    'time', 
    #"day",
    #"hour",
    "user_query_counts_day",
    #"user_query_counts_dayhour",
    #"item_total_query_counts",
    #"item_total_trade_counts",
    "item_total_trade_rate",
    #"user_occupation_id",
    #"user_gender_id",
    #"user_age_level",
    #"user_star_level",
    "user_already_queried_item",

]])
target = ['is_trade']

In [161]:
clf_off = lgb.LGBMClassifier(
        objective='binary',
        # metric='binary_error',
        num_leaves=35,
        depth=8,
        learning_rate=0.05,
        seed=2018,
        colsample_bytree=0.8,
        # min_child_samples=8,
        subsample=0.9,
        n_estimators=20000)

lgb_model_off = clf_off.fit(
        train_off[features],
        train_off[target].values.ravel(),
        eval_set=[(test_off[features], test_off[target].values.ravel())], 
        early_stopping_rounds=200,
        )
print sorted([(features[i], lgb_model_off.feature_importances_[i]) for i in xrange(len(features))], key=lambda x: -int(x[1]))
res = lgb_model_off.predict_proba(test_off[features],)[:, 1]
print log_loss(test_off[target], res)

[1]	valid_0's binary_logloss: 0.647794
Training until validation scores don't improve for 200 rounds.
[2]	valid_0's binary_logloss: 0.606718
[3]	valid_0's binary_logloss: 0.569356
[4]	valid_0's binary_logloss: 0.535289
[5]	valid_0's binary_logloss: 0.504147
[6]	valid_0's binary_logloss: 0.475544
[7]	valid_0's binary_logloss: 0.449214
[8]	valid_0's binary_logloss: 0.42492
[9]	valid_0's binary_logloss: 0.402482
[10]	valid_0's binary_logloss: 0.381652
[11]	valid_0's binary_logloss: 0.36236
[12]	valid_0's binary_logloss: 0.344451
[13]	valid_0's binary_logloss: 0.327773
[14]	valid_0's binary_logloss: 0.312248
[15]	valid_0's binary_logloss: 0.297777
[16]	valid_0's binary_logloss: 0.28428
[17]	valid_0's binary_logloss: 0.271691
[18]	valid_0's binary_logloss: 0.259909
[19]	valid_0's binary_logloss: 0.248887
[20]	valid_0's binary_logloss: 0.238574
[21]	valid_0's binary_logloss: 0.228877
[22]	valid_0's binary_logloss: 0.219834
[23]	valid_0's binary_logloss: 0.211357
[24]	valid_0's binary_logloss

[205]	valid_0's binary_logloss: 0.094693
[206]	valid_0's binary_logloss: 0.0948224
[207]	valid_0's binary_logloss: 0.0949472
[208]	valid_0's binary_logloss: 0.0950724
[209]	valid_0's binary_logloss: 0.0952001
[210]	valid_0's binary_logloss: 0.0953339
[211]	valid_0's binary_logloss: 0.0954385
[212]	valid_0's binary_logloss: 0.0954477
[213]	valid_0's binary_logloss: 0.0955868
[214]	valid_0's binary_logloss: 0.0955701
[215]	valid_0's binary_logloss: 0.0955811
[216]	valid_0's binary_logloss: 0.0955504
[217]	valid_0's binary_logloss: 0.095553
[218]	valid_0's binary_logloss: 0.0956848
[219]	valid_0's binary_logloss: 0.0956773
[220]	valid_0's binary_logloss: 0.095672
[221]	valid_0's binary_logloss: 0.0958062
[222]	valid_0's binary_logloss: 0.095938
[223]	valid_0's binary_logloss: 0.0960644
[224]	valid_0's binary_logloss: 0.0960671
[225]	valid_0's binary_logloss: 0.0960751
[226]	valid_0's binary_logloss: 0.0960741
[227]	valid_0's binary_logloss: 0.0961874
[228]	valid_0's binary_logloss: 0.0961

In [126]:
train_on = train.copy()
test_on = test.copy()
train_on = further_processing_on_train(train_on)
test_on = further_processing_on_test(test_on, train_on)

In [127]:
#train_on['is_trade'] = lbl.fit_transform(train_on["is_trade"])

In [128]:
clf_on = lgb.LGBMClassifier(
        objective='binary',
        # metric='binary_error',
        num_leaves=35,
        depth=8,
        learning_rate=0.05,
        seed=2018,
        colsample_bytree=0.8,
        # min_child_samples=8,
        subsample=0.9,
        n_estimators=lgb_model_off.best_iteration_)

lgb_model_on = clf_on.fit(train_on[features], train_on[target].values.ravel(),)
pred = lgb_model_on.predict_proba(test_on[features])[:, 1]

test_on['predicted_score'] = pred
sub1 = test_on[['instance_id', 'predicted_score']]
sub = pd.read_csv("test.csv", sep=" ")
sub = pd.merge(sub, sub1, on=['instance_id'], how='left')
sub = sub.fillna(0)
sub[['instance_id', 'predicted_score']].to_csv('result/result0420.txt',sep=" ",index=False)

In [129]:
sub['predicted_score'].describe()

count    42888.000000
mean         0.017247
std          0.042277
min          0.005114
25%          0.005278
50%          0.005440
75%          0.005791
max          0.862012
Name: predicted_score, dtype: float64

baseline: 0.09100869727464646


baseline:
Early stopping, best iteration is:
[91]	valid_0's binary_logloss: 0.0824323
[('item_total_trade_rate', 584), ('user_query_counts_day', 330), ('hour', 243), ('user_already_queried_item', 234), ('item_total_query_counts', 168), ('user_query_counts_dayhour', 136), ('item_total_trade_counts', 113), ('user_id', 97), ('context_predict_category_property_2', 86), ('context_predict_category_property_1', 81), ('context_predict_category_property_3', 77), ('user_star', 70), ('context_predict_category_property_4', 62), ('context_predict_category_property_0', 61), ('user_age', 56), ('shop_score_description', 56), ('shop_score_delivery', 52), ('item_property_7', 46), ('item_price_level', 38), ('item_property_9', 33), ('shop_id', 31), ('shop_score_service', 31), ('shop_review_positive_rate', 29), ('item_property_6', 28), ('item_property_len', 27), ('item_property_5', 27), ('item_id', 26), ('item_brand_id', 25), ('item_sales_level', 25), ('item_property_4', 23), ('item_property_8', 23), ('item_category_1', 20), ('context_predict_category_property_len', 20), ('item_property_2', 18), ('item_city_id', 18), ('item_property_3', 16), ('item_property_0', 14), ('item_collected_level', 14), ('item_pv_level', 12), ('shop_review_num_level', 12), ('item_property_1', 10), ('context_page', 10), ('shop_star_level', 10), ('user_gender', 2), ('item_category_2', 0), ('item_category_len', 0), ('user_occupation', 0)]

**0.08243229914644938**

24 full day **0.08156451375758701**

0.08216525571112747